<a href="https://colab.research.google.com/github/luciosjodin/ProcesamientoDelHabla/blob/master/TP3_Desafio_grupo_noticias_Sj%C3%B6dinLucio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TP3-Procesamiento del Habla
##Sjödin Lucio

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[1]

"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks."

In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez ajustado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 8754
print(newsgroups_train.data[idx])


/(hudson)
/If someone inflicts pain on themselves, whether they enjoy it or not, they
/are hurting themselves.  They may be permanently damaging their body.

That is true.  It is also none of your business.  

Some people may also reason that by reading the bible and being a Xtian
you are permanently damaging your brain.  By your logic, it would be OK
for them to come into your home, take away your bible, and send you off
to "re-education camps" to save your mind from ruin.  Are you ready for
that?  





/(hudson)
/And why is there nothing wrong with it?  Because you say so?  Who gave you
/the authority to say that, and set the standard for morality?

Why?

Because: 
I am a living, thinking person able to make choices for myself.
I do not "need" you to show me what you think is the way; I have observed
too many errors in your thinking already to trust you to make up the
rules for me.

Because:
I set the standard for my *own* morality, and I permit you to do 
the same for yourself.  I

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
cossim

array([0.11252759, 0.09561582, 0.17267024, ..., 0.09162675, 0.1121114 ,
       0.03334953])

In [15]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.49040531, 0.48118373, ..., 0.        , 0.        ,
       0.        ])

In [16]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 8754,  6552, 10613, ...,  6988,  6980,  9520])

In [17]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [18]:
mostsim

array([ 6552, 10613,  3616,  8726,  3902])

In [19]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.religion.misc'

In [20]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.religion.misc
talk.religion.misc
talk.religion.misc
talk.politics.mideast
talk.religion.misc


### Modelo de clasificación Naïve Bayes

In [21]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [22]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [23]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío


#**1**.

 Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**No puedes usar la misma solución ya presentada por alguien en el foro antes que Ud. Es decir, sus 5 documentos al azar deben ser diferentes a los ya presentados, o las palabras que elija para el ejercicio 3 deben ser diferentes a las ya presentadas.**



In [33]:
# Imports
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups
import numpy as np

from sklearn.model_selection import ParameterGrid




##**Carga de datos (train/test)**

In [34]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

##**Vectorización**

In [35]:
# Instanciamos un vectorizador
tfidfvect = TfidfVectorizer(stop_words='english') #Eliminamos las stopwords en inglés

In [36]:
# Transformamos los datos en una matriz tf-idf
X_train = tfidfvect.fit_transform(newsgroups_train.data) # Matriz documento-término

# Vectorizamos los textos del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)

# Guardamos los targets que son enteros (etiquetas numéricas de clases)
y_train = newsgroups_train.target

#Y también los targets del conjunto de test
y_test = newsgroups_test.target


In [39]:
# Prints Train

print(type(X_train))
print(f'Forma de la matriz (train): {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')
print("="*100)

# Prints Test
print(type(X_test))
print(f'Forma de la matriz (test): {X_test.shape}')
print(f'cantidad de documentos: {X_test.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_test.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
Forma de la matriz (train): (11314, 101322)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101322
<class 'scipy.sparse._csr.csr_matrix'>
Forma de la matriz (test): (7532, 101322)
cantidad de documentos: 7532
tamaño del vocabulario (dimensionalidad de los vectores): 101322


##**Similaridad de Documentos**

In [42]:
# Selección de la semilla (random) para garantizar que las selecciones aleatorias de numpy sean reproducibles
np.random.seed(66)

# Selección de los 5 documentos aleatorios
documentos = np.random.choice(X_train.shape[0], size=5, replace=False)

print(f"Los documentos seleccionados son (id): {documentos}")

Los documentos seleccionados son (id): [2600 7908 7244 6444 7288]


In [50]:
# Iteración sobre los documentos
for id in documentos:
  print("=" * 100)
  print(f"Documento original (ID: {id}):")
  print("Clase del documento original:", newsgroups_train.target_names[y_train[id]])
  print(f'Extracto: {newsgroups_train.data[id][:300]} ...')
  print("=" * 100)

  # Similaridad Coseno
  cossim = cosine_similarity(X_train[id], X_train)[0]

  # Encontramos los 5 documentos más similares:
  mostsim = np.argsort(cossim)[::-1][1:6]

  # el documento original pertenece a la clase:
  newsgroups_train.target_names[y_train[id]]

  # y los 5 más similares son de las clases:
  for i, id_similar in enumerate(mostsim):
    print(f"\nDocumento similar N°{i+1} (ID: {id_similar}, Similaridad: {cossim[id_similar]:.4f}):")
    print("Clase del documento similar:", newsgroups_train.target_names[y_train[id_similar]])
    print(f'Extracto: {newsgroups_train.data[id_similar][:300]} ...')


Documento original (ID: 2600):
Clase del documento original: alt.atheism
Extracto: 

Anyone who dies for a "cause" runs the risk of dying for a lie.  As for
people being able to tell if he was a liar, well, we've had grifters and
charlatans since the beginning of civilization.  If David Copperfield had
been the Messiah, I bet he could have found plenty of believers.  
Jesus was ha ...

Documento similar N°1 (ID: 10571, Similaridad: 0.1915):
Clase del documento similar: talk.religion.misc
Extracto: : I will clarify my earlier quote.  God's laws were originally written for 
: the Israelites.  Jesus changed that fact by now making the Law applicable to
: all people, not just the Jews.  Gentiles could be part of the kingdom of
: Heaven through the saving grace of God.  I never said that the Law w ...

Documento similar N°2 (ID: 3998, Similaridad: 0.1800):
Clase del documento similar: alt.atheism
Extracto: ->	First I want to start right out and say that I'm a Christian.  It 
->makes sense t

##**Análisis Similaridad de Documentos**

Se seleccionaron 5 documentos al azar del conjunto de entrenamiento para medir su similaridad coseno con el resto de los documentos del conjunto. Veamos cada caso:

**Documento Original (id 2600):**
*   **Clase:** `alt.atheism`

Los 5 documentos más similares, con similaridades entre 0.1674 y 0.1915, provienen en su mayoría de las clases `alt.atheism` y `talk.religion.misc`. Esto claro que tiene sentido, ya que estos grupos de noticias a menudo contienen discusiones relacionadas justamente con el ateísmo, la religión y muchas veces abren debates sobre creencias y cuestiones de fé. La similaridad, aunque no es muy alta en términos absolutos, parece capturar la temática que comparten estos documentos y el original.

**Documento Original (id 7908):**
*   **Clase:** `talk.politics.guns`

Los documentos más similares tienen similaridades entre 0.1892 y 0.2973. En este caso es interesante notar que, si bien hay documentos similares de las clases `talk.politics.misc` y `talk.politics.mideast` (básicamente política), también hay documentos similares de `rec.sport.baseball` y `talk.religion.misc`. La previsualización de estos documentos similares (extractos) a contienen comentarios sobre "sarcasmo". Esto sugiere que la similaridad en este caso podría estar influenciada por el estilo o el tono del texto (el uso del sarcasmo por ejemplo).

**Documento Original (id 7244):**
*   **Clase:** `rec.motorcycles`

Los documentos más similares presentan similaridades de entre 0.1487 y 0.1739. La mayoría de estos documentos similares provienen de la clase `rec.motorcycles`, lo cual es coherente con la clase del documento original (misma clase). Sin embargo, también aparece un documento de `sci.med` y otro de `rec.sport.baseball`. Las similaridades son relativamente bajas, lo que podría indicar que el documento original es algo único dentro de su clase o que la similaridad encontrada consiste en el uso compatido de algunos términos clave que fueron determinantes.

**Documento Original (id 6444):**
*   **Clase:** `rec.sport.baseball`

Los documentos más similares tienen similaridades entre 0.1988 y 0.2756. La mayoría de los documentos similares pertenecen a la clase `rec.sport.baseball`, lo cual es lo esperado. Sin embargo, otra vez, aparece ahora un documento de `alt.atheism` y otro de `sci.electronics`. Esto podría deberse a que estos documentos comparten términos (como el caso anterior) o frases comunes que no son exclusivos del baseball, o a que el documento original toca otros temas.

**Documento Original (id 7288):**
*   **Clase:** `talk.politics.mideast`

Los documentos más similares tienen similaridades entre 0.2731 y 0.2735. Los 5 documentos más similares provienen de la clase `talk.politics.mideast`. Las similaridades son un poco más altas si las comparamos con los otros casos, lo que sugiere que este documento tiene un conjunto de términos y un contexto temático muy específico y recurrente dentro de su grupo de noticias.

##**Conclusión:**

La similaridad coseno basada en la vectorización TF-IDF parece capturar la similitud temática entre documentos, pero también puede verse influenciada por el estilo de escritura, el uso de términos específicos o la tenencia también de términos menos específicos de la clase del documento. De todos modos, en la mayoría de los casos, los documentos más similares tienden a pertenecer a la misma clase que el documento original o a clases temáticamente relacionadas. Sin embargo, la aparición de documentos de clases no relacionadas resalta los desafíos de la similaridad puramente basada en términos y la posibilidad de que términos o frases compartidas puedan generar similaridad incluso entre documentos de diferentes temáticas.

#**2**.
 Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

Matriz Término-Documento/Similaridad de Palabras

In [54]:
# Transposición de la matriz documento-término para obtener la matriz término-documento
X_train_T = X_train.T

# Elección "manual" de las palabras
palabras = ['internet', 'government', 'religion', 'artificial', 'human']

# Obtención de los índices de las palabras seleccionadas en el vocabulario
id_palabras = [tfidfvect.vocabulary_[word] for word in palabras if word in tfidfvect.vocabulary_]

# Iteración sobre las palabras seleccionadas y búsqueda de sus palabras más similares
for word_id in id_palabras:
    word = tfidfvect.get_feature_names_out()[word_id]
    print("=" * 100)
    print(f"Palabra original: {word}")

    # Similaridad Coseno de la palabra con todas las demás palabras
    word_cossim = cosine_similarity(X_train_T[word_id], X_train_T)[0]

    # Obtención de los índices de las palabras más similares (excluyendo la palabra original)
    most_similar_word_indices = np.argsort(word_cossim)[::-1][1:6]

    # Itero e imprimo cada palabra similar
    for i, sim_word_idx in enumerate(most_similar_word_indices):
        sim_word = tfidfvect.get_feature_names_out()[sim_word_idx]
        print(f"Similar {i+1}: {sim_word} (similaridad: {word_cossim[sim_word_idx]:.4f})")

Palabra original: internet
Similar 1: originations (similaridad: 0.2574)
Similar 2: umask (similaridad: 0.2574)
Similar 3: influencing (similaridad: 0.2574)
Similar 4: chfn (similaridad: 0.2574)
Similar 5: constrained (similaridad: 0.2553)
Palabra original: government
Similar 1: libertarian (similaridad: 0.2319)
Similar 2: encryption (similaridad: 0.2264)
Similar 3: agencies (similaridad: 0.2047)
Similar 4: regulation (similaridad: 0.2020)
Similar 5: people (similaridad: 0.2016)
Palabra original: religion
Similar 1: religious (similaridad: 0.2628)
Similar 2: religions (similaridad: 0.2254)
Similar 3: purpsoe (similaridad: 0.2085)
Similar 4: crusades (similaridad: 0.2032)
Similar 5: categorized (similaridad: 0.2009)
Palabra original: artificial
Similar 1: aftereffects (similaridad: 0.4421)
Similar 2: hyperactivity (similaridad: 0.4373)
Similar 3: aggressively (similaridad: 0.3858)
Similar 4: flavoring (similaridad: 0.3830)
Similar 5: extracted (similaridad: 0.3602)
Palabra original: hum

##**Análisis Similaridad de Palabras**

* `'internet'`:

Las palabras más similares (`'originations', 'umask', 'influencing', 'chfn', 'constrained'`) tienen similaridades bajas (alrededor de 0.25). Estas palabras no tienen una conexión semántica obvia con "internet". Esto sugiere que "internet" co-ocurre con una variedad de términos que pueden estar más relacionados con el uso técnico o aspectos específicos discutidos en los grupos de noticias (como "umask" y "chfn" que son términos técnicos e informáticos), en lugar de con sinónimos o términos conceptualmente más cercanos.

* `'government'`:

Las palabras más similares (`'libertarian', 'encryption', 'agencies', 'regulation', 'people'`) tienen similaridades entre 0.20 y 0.23. Como mencioné antes, estas palabras sí tienen una relación conceptual y contextual clara con "government". Esto refuerza la idea de que la similaridad coseno, cuando las palabras aparecen en contextos temáticos definidos, puede capturar relaciones significativas.

* `'religion'`:

Las palabras más similares (`'religious', 'religions', 'purpsoe', 'crusades', 'categorized'`) muestran similaridades de entre 0.20 y 0.26. La relación con "religión" es muy fuerte y directa para la mayoría de ellas, como se analizó previamente. La aparición de 'purpsoe' (posiblemente error tipográfico de 'purpose') y 'categorized' sugiere que las discusiones sobre religión en este dataset a menudo implican la categorización de creencias o la discusión de propósitos religiosos.

* `'artificial'`:

Las palabras más similares (`'aftereffects', 'hyperactivity', 'aggressively', 'flavoring', 'extracted'`) tienen similaridades notablemente más altas (entre 0.36 y 0.44) en comparación con las otras palabras. Esto sugiere que "artificial" tiene un contexto de uso más específico y recurrente en el dataset. Las palabras similares apuntan fuertemente a discusiones relacionadas con aditivos alimentarios, salud, y procesos (como extracción o manipulación), lo que probablemente proviene de los grupos de noticias relacionados con ciencia o medicina.

* `'human'`:

Las palabras más similares (`'beings', 'wsidom', 'davar', 'adressed', 'nestorianism'`) tienen similaridades bajas (alrededor de 0.22 a 0.24). "Beings" es semánticamente muy cercana a "human". Las otras palabras ('wsidom', 'davar', 'adressed', 'nestorianism') parecen menos relacionadas a primera vista. 'Nestorianism' es una rama del cristianismo, lo que podría vincularse a discusiones sobre la naturaleza humana en contextos religiosos. Las otras podrían ser errores tipográficos o términos muy específicos de algún subdominio.

## **Conclusión:**

La similaridad coseno basada en TF-IDF refleja la co-ocurrencia de palabras. Para términos que aparecen en contextos temáticos muy específicos y recurrentes (como 'artificial' en relación con aditivos o 'religion' y 'baseball' con sus términos asociados), la similaridad puede ser más alta y semánticamente interpretable. Para términos más generales o que aparecen en contextos más dispersos ('internet', 'human'), las similaridades pueden ser más bajas y las palabras similares menos relacionadas semánticamente, reflejando patrones de co-ocurrencia más sutiles o variados en el dataset.

#**3**.
 Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

Modelo de clasificación Naïve Bayes

In [55]:
# Parámetros para TfidfVectorizer
param_grid_tfidf = {
    'max_df': [0.95, 0.98],
    'min_df': [2, 3],
    'ngram_range': [(1, 1), (1, 2)],
    # 'use_idf': [True, False] # Eliminamos use_idf según la solicitud
}

# Parámetros para los modelos Naïve Bayes
param_grid_nb = {
    'alpha': [0.1, 0.5, 1.0] # Incluimos el suavizado de Laplace (alpha)
}       # alpha es un parámetro de suavizado de Laplace. Sirve para evitar probabilidades cero
        # cuando una palabra que aparece en el test no estaba en el entrenamiento,
        # sumando 'alpha' a los conteos de términos.

# Inicialización de variables
best_score = 0
best_params_tfidf = {}
best_params_nb = {}
best_model = None

# Iteraración sobre el grid de parámetros del vectorizador
for params_tfidf in ParameterGrid(param_grid_tfidf):
    print(f"Probando parámetros del vectorizador: {params_tfidf}")

    # Instanciar TfidfVectorizer con los parámetros actuales
    tfidfvect_tuned = TfidfVectorizer(**params_tfidf, stop_words='english')

    # Ajustar el vectorizador a los datos de entrenamiento y transformar
    X_train_tuned = tfidfvect_tuned.fit_transform(newsgroups_train.data)

    # Transformar los datos de prueba
    X_test_tuned = tfidfvect_tuned.transform(newsgroups_test.data)

    # Iterar sobre el grid de parámetros de los modelos Naïve Bayes
    for params_nb in ParameterGrid(param_grid_nb):
        print(f"  Probando parámetros del mdelo Naïve Bayes: {params_nb}")

        # Multinomial

        clf_mnb = MultinomialNB(**params_nb)
        clf_mnb.fit(X_train_tuned, y_train)
        y_pred_mnb = clf_mnb.predict(X_test_tuned)
        f1_macro_mnb = f1_score(y_test, y_pred_mnb, average='macro')

        print(f"    - MultinomialNB F1-score macro: {f1_macro_mnb:.4f}")

        if f1_macro_mnb > best_score:
            best_score = f1_macro_mnb
            best_params_tfidf = params_tfidf
            best_params_nb = params_nb
            best_model = 'MultinomialNB'

        # ComplementNB
        clf_cnb = ComplementNB(**params_nb)
        clf_cnb.fit(X_train_tuned, y_train)
        y_pred_cnb = clf_cnb.predict(X_test_tuned)
        f1_macro_cnb = f1_score(y_test, y_pred_cnb, average='macro')

        print(f"    - ComplementNB F1-score macro: {f1_macro_cnb:.4f}")

        if f1_macro_cnb > best_score:
            best_score = f1_macro_cnb
            best_params_tfidf = params_tfidf
            best_params_nb = params_nb
            best_model = 'ComplementNB'

print("=" * 50)
print(f"Mejor F1-score macro obtenido: {best_score:.4f}")
print(f"Mejores parámetros del vectorizador: {best_params_tfidf}")
print(f"Mejores parámetros del modelo: {best_params_nb}")
print(f"Mejor modelo: {best_model}")

Probando parámetros del vectorizador: {'max_df': 0.95, 'min_df': 2, 'ngram_range': (1, 1)}
  Probando parámetros del mdelo Naïve Bayes: {'alpha': 0.1}
    - MultinomialNB F1-score macro: 0.6798
    - ComplementNB F1-score macro: 0.6887
  Probando parámetros del mdelo Naïve Bayes: {'alpha': 0.5}
    - MultinomialNB F1-score macro: 0.6626
    - ComplementNB F1-score macro: 0.6974
  Probando parámetros del mdelo Naïve Bayes: {'alpha': 1.0}
    - MultinomialNB F1-score macro: 0.6512
    - ComplementNB F1-score macro: 0.6943
Probando parámetros del vectorizador: {'max_df': 0.95, 'min_df': 2, 'ngram_range': (1, 2)}
  Probando parámetros del mdelo Naïve Bayes: {'alpha': 0.1}
    - MultinomialNB F1-score macro: 0.6822
    - ComplementNB F1-score macro: 0.7018
  Probando parámetros del mdelo Naïve Bayes: {'alpha': 0.5}
    - MultinomialNB F1-score macro: 0.6598
    - ComplementNB F1-score macro: 0.7101
  Probando parámetros del mdelo Naïve Bayes: {'alpha': 1.0}
    - MultinomialNB F1-score macr

##**Datos de los Modelos Naïve Bayes**

Para entrenar modelos de clasificación Naïve Bayes y maximizar su desempeño (medido por el F1-score macro) en el conjunto de datos de test, se realizó una búsqueda de hiperparámetros explorando diferentes configuraciones para el `TfidfVectorizer` y los modelos `MultinomialNB` y `ComplementNB`.

Los parámetros explorados para el `TfidfVectorizer` fueron:
*   `max_df`: [0.95, 0.98] (ignorar términos que aparecen en una proporción mayor a este umbral)
*   `min_df`: [2, 3] (ignorar términos que aparecen en una proporción menor a este umbral)
*   `ngram_range`: [(1, 1), (1, 2)] (considerar unigramas o unigramas y bigramas)
*   `stop_words='english'`: Se eliminaron las stop words comunes en inglés.

Para los modelos Naïve Bayes (`MultinomialNB` y `ComplementNB`), se exploró el parámetro de suavizado de Laplace:
*   `alpha`: [0.1, 0.5, 1.0] (valor de suavizado, ayuda a manejar términos desconocidos en el conjunto de prueba)



##**Análisis de los resultados:**

Cito: **Mejor F1-score macro obtenido: 0.7101**

Este desempeño óptimo se logró con la siguiente combinación de parámetros:
*   **Vectorizer (`TfidfVectorizer`) Parámetros:**
    *   `max_df`: 0.95
    *   `min_df`: 2
    *   `ngram_range`: (1, 2)
*   **Modelo Naïve Bayes Parámetros:**
    *   `alpha`: 0.5
*   **Mejor Modelo:** ComplementNB

**Reflexión sobre el mejor desempeño:**

El hecho de que la mejor puntuación se alcanzara con `ngram_range=(1, 2)` sugiere que incluir bigramas (pares de palabras) en la vectorización, además de unigramas (palabras individuales), fue beneficioso. Los bigramas pueden capturar información contextual valiosa que las palabras individuales no siempre proporcionan por sí solas, como frases comunes o términos técnicos específicos de ciertos grupos de noticias (por ejemplo, "operative system", "motorcycle parts").

Los parámetros `max_df=0.95` y `min_df=2` para el vectorizador son razonables. Excluir términos extremadamente comunes (presentes en más del 95% de los documentos) ayuda a eliminar palabras que probablemente no sean discriminatorias entre clases, mientras que excluir términos muy raros (presentes en menos de 2 documentos) ayuda a reducir el ruido y la dimensionalidad, y a manejar mejor los posibles errores tipográficos existentes.

El uso de `alpha=0.5` para el suavizado de Laplace indica que aplicar un suavizado moderado fue efectivo. El suavizado de Laplace es crucial en Naïve Bayes para evitar que las probabilidades de las clases se vuelvan cero cuando un término del conjunto de prueba no aparece en el conjunto de entrenamiento. Un valor de 0.5 parece haber funcionado mejor que 0.1 o 1.0 en este caso particular.



##**Conclusión:**

Finalmente, el modelo **ComplementNB** obtuvo el mejor rendimiento general en comparación con MultinomialNB. ComplementNB a menudo funciona mejor que MultinomialNB en conjuntos de datos con clases desequilibradas, ya que se centra en el complemento de cada clase. Esto sugiere que, a pesar de tener 20 clases, puede haber cierto grado de desequilibrio en la distribución de documentos entre los grupos de noticias en este dataset, lo que favoreció el enfoque de ComplementNB.

La combinación ganadora de un vectorizador que considera unigramas y bigramas con un manejo adecuado de términos frecuentes/raros, junto con un suavizado de Laplace óptimo en el modelo ComplementNB, permitió alcanzar el mejor rendimiento de clasificación para este dataset.